In [ ]:
import os 
import re
import random
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns

from pathlib import Path
from Bio import SeqIO
from itertools import islice
from tqdm import tqdm

import torch
import tensorflow as tf
from keras.utils.np_utils import to_categorical
from keras.models import Model
from keras.layers import Conv2D, Dropout, Input
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.preprocessing import LabelEncoder

%cd dataset/

# Preprocessing

## Embedding generation

### Core PU Sequence generation

In [ ]:
classes = []
topo = []
protofold = []
fq = []
pu = []

with open('./LISTS_CATFS_PU_PROTOTYPE_with_fused_famillies', 'r') as file:
    for line in islice(file, 7, None):
        tmp = line.split(maxsplit=5)
        if len(tmp[2])>=2 and int(tmp[4])>10:
            classes.append(tmp[0])
            topo.append(tmp[2])
            protofold.append(tmp[3])
            fq.append(tmp[4])
            pu.append(tmp[5])

x = {'Class':classes, 'Topology':topo, 'Protofold':protofold, 'Frequency':fq, 'PU':pu}
list_pu = pd.DataFrame(x, columns=['Class', 'Topology', 'Protofold', 'Frequency', 'PU'])
list_pu = list_pu.astype({'Frequency': 'int'})

Sort Dataframe by Frequency and
Remove duplicate for Protofold

In [ ]:
list_pu = list_pu.sort_values('Frequency', ascending=False)
filt_pu = list_pu.drop_duplicates(subset=['Topology', 'Protofold','Frequency'], keep='first')
filt_pu = filt_pu.reset_index(drop=True)

Generate list of pu without redundancy of pu in a protofold class (for example, 30.g219 and 50.g219 are considered the same so if a pu is present in both class, only the first one will be kept)


In [ ]:
df_pu = pd.DataFrame().assign(Protofold = filt_pu.iloc[:300,:]['Protofold'], PU = filt_pu.iloc[:300,:]['PU'])
tmp_df = df_pu['PU'].str.split(' ').apply(pd.Series, 1).stack() #Split PU into rows
tmp_df = tmp_df.replace(r'n','', regex=True).replace(r':','', regex=True) #Remove 'n' and ':'
tmp_df.index = tmp_df.index.droplevel(-1)
tmp_df.name = 'PU' #Need to rename to have index with which joining can be done
del df_pu['PU']
df_pu = df_pu.join(tmp_df)
df_pu['Class'] = df_pu['Protofold'].str.split('.', expand = True)[1] #Get ID : 35.g129 -> g219
df_pu = df_pu.drop_duplicates(subset=['PU', 'Class'], keep='first')
df_pu

In [ ]:
select_pu, unique_pu = [], []
select_pu = df_pu[['Protofold', 'PU']].values.tolist()
[unique_pu.append([proto, pu]) for proto, pu in select_pu if pu not in [item[1] for item in unique_pu]]
print(fThere are {len(select_pu)} total PU and {len(unique_pu)} unique)

### Generate Core PU Embedding with ESMFold2

Generate PU for aligning and getting source sequence. Embedding will be done on unique_pu first as some of their source sequences are shared with select_pu

In [ ]:
def pdb_gen(file):
    with open(tmp.txt) as file:
      f = file.read()
      pdb = re.search(r(>>w{4}_.), f).group(0)[2:]
      pos = re.findall(rd+-d+, f)[1]
      yield pdb, pos

In [ ]:
no_result = []

for proto, pu in tqdm(islice(unique_pu, len(unique_pu))):
  name = f'{pu[0:4].lower()}_{pu[4]}'
  !ssearch36 ./PU/$pu.fasta ./pdb_seqres.txt | grep -A2 -m1 ^>>$name > ./tmp.txt
  
  if os.stat(tmp.txt).st_size != 0: #If file not empty = If alignment gives result
    for y in pdb_gen(tmp.txt):
      pdb, pos = y
      !grep -A1 $pdb ./pdb_seqres.txt | sed s/>{pdb}/&_{proto}_{pos}/ >> pdb_core.fasta
  else:
    no_result.append([proto, pu])

In [ ]:
with open('tmp.fasta', 'w') as f:
    records = SeqIO.parse('./pdb_core.fasta', 'fasta')
    for record in records:
        f.write(f'>{record.id}n')
        f.write(f'{record.seq}n')
!esm-extract esm2_t33_650M_UR50D tmp.fasta ./emb_esm2/ --include per_tok

List of PU without embedding, including those in select_pu

In [ ]:
l = []
for proto, pu in unique_pu:
    l.append([pu, proto, f'{pu[:4].lower()}_{pu[4]}_{proto}'])

l_emb = []
with open('list_emb.txt') as f:
    lines = f.readlines()
    for line in lines:
        x = line.split('_')
        l_emb.append(f'{x[0]}_{x[1]}_{x[2]}')

no_emb = []
[no_emb.append([pu, p, x]) for pu, p, x in l if x not in l_emb]

## Data generation

In [ ]:
def pad(array, target_shape):
    return np.pad(array, [(0, target_shape[i] - array.shape[i]) for i in range(len(array.shape))], constant,)

def data_gen(path_file, path_save, name, pos_start, pos_end):
    tmp = torch.load(path_file)
    tens = np.array(tmp[representations][33])[pos_start:pos_end]

    try:
        tens_padded = pad(tens, (60, 1280))
    except ValueError:
        tens_padded = np.resize(tens, (60, 1280))
    
    if not os.path.exists(path_save):
        os.makedirs(path_save)
        np.savetxt(f'{path_save}/{name}.csv', tens_padded, delimiter = ,)
    else:
        np.savetxt(f'{path_save}/{name}.csv', tens_padded, delimiter = ,)

### PU cores

Make a list of all embedding done

In [ ]:
list_files = os.listdir(emb_esm2/)

In [ ]:
for file in tqdm(list_files):
    name = f{os.path.basename(file).split('_')[0]}_{os.path.basename(file).split('_')[1]}
    pu_class = os.path.basename(file).split('_')[2]
    pos_start = int(file.split('_')[-1].split(-)[0])-1
    pos_end = int(file.split('_')[-1].split('-')[-1].rsplit(.)[0])

    path_save = f/home/arnaud/projet_long/dataset/core/{pu_class}
    path_file = f/home/arnaud/projet_long/emb_esm2/{file}
    
    if os.path.isfile(f{path_save}/{name}.csv):
        pass
    else:
        data_gen(path_file, path_save, name, pos_start, pos_end)

### Noncores PU

For Noncore PUs
- Directly generate padded data from embedded source sequences used for core PUs
- If PU don't have their source sequences embedded, we generate them 
- Generate data for those PU

1) Generate list of noncores PU

In [ ]:
nc_list_file = []
nc_names = []
nc_length = []
nc_pos_start = []
nc_pos_end = []
with open('LIST_OF_PUs_UNCLASSED.txt', 'r') as file:
    for line in file: #Ex : 16VPA_10_41_59.fasta
        tmp = line.rstrip().split('_')[0]
        nc_names.append(f{tmp[:-1].lower()}_{tmp[-1]})
        nc_length.append(line.split('_')[1])
        nc_pos_start.append(int(line.rstrip().split('_')[2]))
        nc_pos_end.append(int(line.rstrip().split('_')[3].split('.')[0]))
        nc_list_file.append(line.rstrip().split('.')[0])

In [ ]:
x = {Filename : nc_names, 'Length' : nc_length, Start : nc_pos_start,'End' : nc_pos_end}
df_nc = pd.DataFrame(x, columns=[Filename, Length, Start, End])
df_nc = df_nc.astype({'Length': 'int'})

df_nc = df_nc.sort_values('Length', ascending=False)
df_modif_nc = df_nc.drop_duplicates(subset=[Filename, Length, Start, End], keep='first')
df_modif_nc = df_modif_nc[df_modif_nc['Length'] >= 10]
df_modif_nc = df_modif_nc.reset_index(drop=True)
df_modif_nc

In [ ]:
nc_data = df_modif_nc.replace(r'_','', regex=True).astype(str).values.tolist()
nc_data = ['_'.join(sublist).upper() for sublist in nc_data]

2) Generate padded data for those with source sequences embedded

In [ ]:
nc_no_emb = []
for filename in tqdm(nc_data):
    name = filename.split('_')[0][:-1].lower() # To match embedding file name : 2X6WA_27_175_199 in nc_data -> 2x6w* in embedding file
    for file in glob.glob(f'/home/arnaud/projet_long/emb_esm2/{name}*'):
        save = /home/arnaud/projet_long/dataset/noncore/
        nc_start = int(filename.split('_')[-2])
        nc_end = int(filename.split('_')[-1])
        data_gen(file, save, filename, nc_start, nc_end)
    tmp = f{save}/{filename}.csv
    if os.path.isfile(tmp):
        pass
    else:
        nc_no_emb.append(filename)

3) From list of PU without source sequences embedding, generate embedding with ESM-extract

In [ ]:
extra = []
tmp = [x for x in [f{items[:4].lower()}_{items[4]} for items in nc_no_emb] if x not in extra]
extra = list(set(tmp))
print(len(extra))

In [ ]:
with open('tmp.fasta', 'w+') as f:
    for element in tqdm(extra):    
        records = SeqIO.parse('./pdb_seqres.txt', 'fasta')
        for record in records:
            if element in record.id:
                f.write(f'>{record.id}n')
                f.write(f'{record.seq}n')
!esm-extract esm2_t33_650M_UR50D tmp.fasta ./emb_nc_extra/ --include per_tok

Some PU don't have their source sequence in the database. In this case, we won't be using PUs associated to these sequences.

In [ ]:
l = os.listdir("./emb_nc_extra/")
nc_emb_miss = [file for file in extra if not any(file in emb for emb in l)]
len(nc_emb_miss)

4) Finally, generate padded for PU left over

In [ ]:
for filename in tqdm(nc_no_emb): #List of PU without embedding generated previously
    name = filename.split('_')[0][:-1].lower() # To match embedding file name : 2X6WA_27_175_199 in nc_data -> 2x6w* in embedding file
    for file in glob.glob(f'/home/arnaud/projet_long/emb_nc_extra/{name}*'):
        save = "/home/arnaud/projet_long/dataset/noncore/"
        nc_start = int(filename.split('_')[-2])
        nc_end = int(filename.split('_')[-1])
        data_gen(file, save, filename, nc_start, nc_end)

# Dataset preparation

## List preparation

In [ ]:
def split_data(split_coeff, data_list, train_data, test_data, val_data):
    data_split = int((len(data_list))*split_coeff)
    test_data += data_list[data_split:]

    tmp_train_data = data_list[:data_split] # 80% of initial data
    train_split = int(len(tmp_train_data)*split_coeff)

    train_data += tmp_train_data[:train_split]
    val_data += tmp_train_data[train_split:]
    return train_data, test_data, val_data

Select 80% of data of each PU protofold for training and remaining 20% for test, and further split the initial 80% of training data into 80/20 for training and validation. The overall percentage of initial data will be :
- 64% for training
- 20% for testing
- 16% for validation

In [ ]:
data_count = {}
train_data = []
test_data = []
val_data = []
path = '/home/arnaud/projet_long/dataset/core_test'
core_list = os.listdir(path)
split_coeff = .8
for core in core_list:
    data_list = [f"core/{core}/{data}" for data in os.listdir(os.path.join(path, core))]
    random.shuffle(data_list)
    split_data(split_coeff, data_list, train_data, test_data, val_data)
    data_count[core] = len(data_list)

In [ ]:
nc_path = '/home/arnaud/projet_long/dataset/noncore_test'
nc_list = [f"noncore/{data}" for data in os.listdir(nc_path)]
split_data(split_coeff, nc_list, train_data, test_data, val_data)
data_count['noncore'] = len(nc_list)

Create data for testing

In [ ]:
import shutil
path = '/home/arnaud/projet_long/dataset/core'
core_list = os.listdir(path)
coef = int((len(core_list)+1)*.5)
for core in core_list:
    data_list = [f"core/{core}/{data}" for data in os.listdir(os.path.join(path, core))]
    random.shuffle(data_list)
    path_target = f'/home/arnaud/projet_long/dataset/core_test/{core}/'
    for element in data_list[:coef]:
        if not os.path.exists(path_target):
            os.makedirs(path_target)
        shutil.copy(element, path_target)

In [ ]:
path_target = '/home/arnaud/projet_long/dataset/noncore_test'
nc_path = '/home/arnaud/projet_long/dataset/noncore'
nc_list = [f"noncore_test/{data}" for data in os.listdir(nc_path)]
coef = int((len(nc_list)+1)*.5)
random.shuffle(nc_list)
for element in nc_list[:coef]:
    shutil.copy(element, path_target)

### Details on data generated

In [1]:
print(f"After splitting data between train/test dataset,nTrain dataset contains {len(train_data)} PUsnTest dataset contains {len(test_data)} PUsnVal dataset contains {len(val_data)} PUs")

NameError: name 'train_data' is not defined

In [ ]:
a = len(train_data) + len(test_data) + len(val_data)
print(f"Total is {a} PUs")
print(f"Train percentage = {round(len(train_data)*100/a)}% of total PUs")
print(f"Test percentage = {round(len(test_data)*100/a)}% of total PUs")
print(f"Val percentage = {round(len(val_data)*100/a)}% of total PUs")

In [ ]:
pu = [key for key in data_count.keys() if key != 'noncore']
value = [value for key, value in data_count.items() if key != 'noncore']

plt.figure(figsize=(15,7))
plt.bar(range(len(data_count)-1), value, tick_label = pu)
plt.yticks(size = 10)
plt.xlabel("Protofold classes", size = 10)
plt.ylabel("Number of PUs", size = 10)
plt.title(f"Barplot of core PUs data, n_class = {len(data_count.keys())-1}", size = 10)
plt.show()

In [ ]:
binary_key = ['Core', 'Non Core']
binary_value = [sum(value), data_count['noncore']]

plt.figure(figsize=(3, 5))
plt.bar(binary_key, binary_value, tick_label = binary_key)
plt.yticks(size = 8)
plt.xlabel("Core PU/Non Core PU", size = 8)
plt.ylabel("Number of PUs", size = 8)
plt.title(f"Barplot of core/non core PU", size = 8)
plt.show()

## Train, Val, Test dataset generation

In [ ]:
def data_generator(files, batch_size, num_classes, method):
    num_files = len(files)
    label_encoder = LabelEncoder()
    
    while True:
        np.random.shuffle(files)
        for i in range(0, num_files, batch_size):
            batch_files = files[i:i+batch_size]
            batch_data = []
            batch_labels = []
            for file in batch_files:
                data = pd.read_csv(file, header=None).values
                batch_data.append(data)
                label = tf.strings.split(file, os.path.sep)[-2]
                if method == "binary":
                    if label == 'noncore':
                        batch_labels.append(0)
                    else:
                        batch_labels.append(1)
                else:
                    batch_labels.append(label)
            
            batch_data = np.array(batch_data)
            if method == "binary":
                if label == 'noncore':
                    batch_labels = np.array(batch_labels)
                    batch_labels = to_categorical(batch_labels, 2)
                else:
                    batch_labels = label_encoder.fit_transform(batch_labels)  # Encode labels as integers
                    batch_labels = to_categorical(batch_labels, num_classes)
            
            yield batch_data, batch_labels

For data : 
- data_count.keys() containing all protofold classes
- train_data and test_data containing list of files for the generator

In [ ]:
batch_size = 128
num_classes = len(data_count.keys())

random.shuffle(train_data)
random.shuffle(val_data)
random.shuffle(test_data)
train_generator = data_generator(train_data, batch_size, num_classes, method = "binary")
val_generator = data_generator(val_data, batch_size, num_classes, method = "binary")
test_generator = data_generator(test_data, batch_size, num_classes, method = "binary")

# Network

## CNN (differenciation between PU/not PU)

In [ ]:
def cnn():
  inputs = Input(shape=(60, 1280, 1))
  conv_1 = Conv2D(64,(6,9), activation = "relu")(inputs)
  pool_1 = MaxPooling2D(pool_size = (3,3))(conv_1)
  conv_2 = Conv2D(32,(6,9), activation = "relu")(pool_1)
  pool_2 = MaxPooling2D(pool_size = (3,3))(conv_2)
  flat = Flatten()(pool_2)
  dense_1 = Dense(activation = 'relu', units = 128)(flat)
  drop = Dropout(0.2)(dense_1)
  dense_2 = Dense(activation = 'softmax', units = 1)(drop)
  model = Model(inputs=inputs, outputs=dense_2)
  return model

early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=3,
    mode='min',
    verbose=1)
model = cnn()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

In [ ]:
steps_per_epoch = len(train_data) // batch_size
cnn_model = model.fit(train_generator, validation_data=val_generator, steps_per_epoch = steps_per_epoch, epochs=25, callbacks=[early_stopping_callback])
model.save('./cnn_save/model.h5')

##